# 좌표계 변환

In [1]:
from pyproj import Proj, transform
import numpy as np
import pandas  as pd

카카오API에서 제공하는 좌표계는 `EPSG:5181`

이를 서울시빅데이터캠퍼스에서 제공하는 데이터인 `utm-k` 좌표계로 변환

In [2]:
pd.set_option('display.float_format', '{:.14f}'.format)

## 데이터 불러오기
- 2015, 2016 축제 정보 데이터 불러오기

In [3]:
info = pd.read_excel("festival_info_1516_final.xlsx", convert_floats=False)

In [4]:
info

,축제명_수정,시작일,종료일,축제장소_수정,축제설명_수정,관련 누리집,문의_수정,부대행사,소요시간,소요예산(천원),...,주최/주관기관,축제성격,축제참가인원(명),address_name,x,y,road_address_name,place_url,개최년도,축제일수
0,서울국제빵과자페스티벌,2015-11-18,2015-11-21,코엑스,2015년 11월 18일부터 21일까지 코엑스에서 <서울국제빵과자페스티벌>이 열린다...,http://siba-expo.com,(주)코엑스 02-6000-8164,"- 엄마와 함께 예쁜 케이크, 쿠키 만들기\r- 대한민국 동네빵집 페스티벌\r- 커...",NaN,NaN,...,(사)대한제과협회/(주)코엑스,경연사업,NaN,서울 강남구 삼성동 159,127.05879841161699,37.51263076118840,서울 강남구 영동대로 513,http://place.map.kakao.com/26437384,2015,4
1,아르헨티나페스티발,2015-09-05,2015-09-05,대학로 마로니에공원,사단법인 한국아르헨티나협회와 주한 아르헨티나 대사관은 한인 이민 50주년을 기념하기...,NaN,사단법인 한국아르헨티나협회 02-3442-5592,NaN,NaN,NaN,...,사단법인 한국아르헨티나협회/주한 아르헨티나대사관,문화예술,NaN,서울 종로구 동숭동 1-124,127.00282574530300,37.58028876432680,서울 종로구 대학로 104,http://place.map.kakao.com/10353868,2015,1
2,한강홀릭,2015-09-19,2015-09-19,반포한강공원,"한강을 그리다! 한강에 열광하다!\r한강에 홀딱 빠져드는 이색축제,\rHangang...",http://www.hangangholic.com/,2015 한강홀릭사무국 02-449-9060,"- 이색레저스포츠체험: 열기구체험, 요트체험, 행글라이딩 체험, 유로번지 체험, 익...",NaN,NaN,...,문화체육관광부 / 한국관광공사,기타,NaN,서울 서초구 반포동 1331,126.98847306834401,37.50544760752040,NaN,http://place.map.kakao.com/24580371,2015,1
3,그랜드케이팝페스티벌,2015-09-04,2015-09-04,잠실 올림픽 주 경기장,대한민국의 최정상급 케이팝 스타들이 대거 출연하는 케이팝 한류 페스티벌이 9월4일 ...,http://www.grandkpopfestival.com,"한국관광공사 관리자 033-738-3224, 3226",NaN,NaN,NaN,...,문화체육관광부,문화예술,NaN,서울 송파구 잠실동 10-1,127.07302486580099,37.51459061509421,서울 송파구 올림픽로 25,http://place.map.kakao.com/8530688,2015,1
4,서리풀페스티벌,2015-09-15,2015-09-20,예술의 전당,서리풀페스티벌은 서초구의 고품격 예술자원과 글로벌 문화콘텐츠를 결집한 축제로 K-컬...,http://blog.naver.com/sccf150508,"서리풀페스티벌 조직위원회 관리자 02-2155-8212, 3477-2856",[서초 명사와 함께 하는 토요벼룩시장]\r - 일시 및 장소 : 9.19(토) 1...,NaN,NaN,...,"(재)서초문화재단, 서초구",문화예술,NaN,서울 서초구 서초동 134-1,127.01362582484900,37.47943350201761,서울 서초구 남부순환로 2406,http://place.map.kakao.com/25466462,2015,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,세종탄신기념문화행사,2016-05-15,2016-05-15,국립한글박물관,제 619돌을 맞은 세종대왕 탄신 기념 문화행사가 5월 15일 국립한글박물관 잔디마...,http://www.hangeul.go.kr/event/userEventApplyD...,02-2124-6200,"- 공연 \r • 음악극 ‘위대한 성군, 세종’\r• 음악을 사랑한 세종(궁중음...",NaN,NaN,...,국립한글박물관,문화예술,NaN,서울 용산구 용산동6가 168-6,126.98097747372800,37.52118537749170,서울 용산구 서빙고로 139,http://place.map.kakao.com/25612651,2016,1
72,희망의운동화나눔축제,2016-05-20,2016-05-21,서울 광화문 광장,<희망의 운동화 나눔 축제>는 운동화 위에 세계 평화와 희망을 상징하는 이미지를 그...,http://www.mizy.net,서울시립청소년문화교류센터(Mizy) 관리자 미지(Mizy)센터 02-755-1024,NaN,기간내자유,NaN,...,"서울특별시, 대산문화재단, 관세청",기타,NaN,서울 종로구 세종로 1-68,126.97691639601514,37.57260286081019,NaN,http://place.map.kakao.com/8193961,2016,2
73,빗물축제,2016-07-28,2016-07-31,서울시청 광장,물순환 시민 문화제에서 서울시와 시민들의 실천의지가 함께 할 수 있는 축제의 장으로...,https://www.facebook.com/events/1126526240721644,관리자 물순환정책과 02-2133-3762,"빗물 퍼포먼스(플레쉬몹), , 물순환 박람회, 레인마켓, 빗물놀이터, 물순환 시민참...",12시간,NaN,...,서울시,생태자원,NaN,서울 중구 태평로1가 54-3,126.97796892695699,37.56567260428050,NaN,http://place.map.kakao.com/7940285,2016,4
74,남산골한옥마을세시맞이오(五)대감설날잔치,2016-02-08,2016-02-09,서울특별시 중구 퇴계로34길 28,서울 중구에 위치한 남산골한옥마을에서는 설 명절을 맞아 다양한 전통문화 프로그램을 ...,http://hanokmaeul.or.kr,02-2264-4412,NaN,기간 내 자유,NaN,...,서울특별시/남산골한옥마을,전통역사,NaN,서울 중구 필동2가 84-1,126.99475754194501,37.55927386383180,서울 중구 퇴계로34길 28,http://place.map.kakao.com/8246127,2016,2


In [5]:
info[['x', 'y']].iloc[0]

x   127.05879841161699
y    37.51263076118840
Name: 0, dtype: float64

In [6]:
# Projection 정의
# UTM-K
proj_UTMK_5178 = Proj(init='epsg:5178') # UTM-K(Bassel) 도로명주소 지도 사용 중
proj_UTMK_5179 = Proj(init='epsg:5179') # UTM-K(Bassel) 도로명주소 지도 사용 중

# epsg:5181
proj_epsg_5181 = Proj(init='epsg:5181') # epsg:5181

proj_WGS84 = Proj(init='epsg:4326') # Wgs84 경도/위도, GPS사용 전지구 좌표

In [7]:
#x1, y1 = 126.9784147, 37.5666805
x1, y1 = 127.05879841161699, 37.51263076118840
x2, y2 = transform(proj_WGS84, proj_UTMK_5178, x1, y1)
x2, y2 = transform(proj_WGS84, proj_epsg_5181, x1, y1)
print(x2,y2)

205198.40000003946 445907.60000078374


In [8]:
def transform_epsg_5181_to_utmk(df):
    return pd.Series(transform(proj_WGS84, proj_UTMK, df['x'], df['y']))#, index=['x', 'y'])

In [10]:
# epsg:5181 -> UTM-K 샘플
tmp = info.copy()

# x, y 컬럼을 이용하여 UTM-K좌표를 WGS84로 변환한 Series데이터 반환
def transform_epsg_5181_to_utmk_5178(df):
    return pd.Series(transform(proj_WGS84, proj_UTMK_5178, df['x'], df['y']))#, index=['x', 'y'])

def transform_epsg_5181_to_utmk_5179(df):
    return pd.Series(transform(proj_WGS84, proj_UTMK_5179, df['x'], df['y']))#, index=['x', 'y'])


tmp[['x_5178', 'y_5178']] = tmp.apply(transform_epsg_5181_to_utmk_5178, axis=1)
tmp[['x_5179', 'y_5179']] = tmp.apply(transform_epsg_5181_to_utmk_5179, axis=1)

In [11]:
tmp

,축제명_수정,시작일,종료일,축제장소_수정,축제설명_수정,관련 누리집,문의_수정,부대행사,소요시간,소요예산(천원),...,x,y,road_address_name,place_url,개최년도,축제일수,x_5178,y_5178,x_5179,y_5179
0,서울국제빵과자페스티벌,2015-11-18,2015-11-21,코엑스,2015년 11월 18일부터 21일까지 코엑스에서 <서울국제빵과자페스티벌>이 열린다...,http://siba-expo.com,(주)코엑스 02-6000-8164,"- 엄마와 함께 예쁜 케이크, 쿠키 만들기\r- 대한민국 동네빵집 페스티벌\r- 커...",NaN,NaN,...,127.05879841161699,37.51263076118840,서울 강남구 영동대로 513,http://place.map.kakao.com/26437384,2015,4,961198.04308944870718,1945712.96791886328720,961008.62817778426688,1946019.03093189164065
1,아르헨티나페스티발,2015-09-05,2015-09-05,대학로 마로니에공원,사단법인 한국아르헨티나협회와 주한 아르헨티나 대사관은 한인 이민 50주년을 기념하기...,NaN,사단법인 한국아르헨티나협회 02-3442-5592,NaN,NaN,NaN,...,127.00282574530300,37.58028876432680,서울 종로구 대학로 104,http://place.map.kakao.com/10353868,2015,1,956290.94805810670368,1953243.91232858435251,956101.68447418534197,1953549.98817524639890
2,한강홀릭,2015-09-19,2015-09-19,반포한강공원,"한강을 그리다! 한강에 열광하다!\r한강에 홀딱 빠져드는 이색축제,\rHangang...",http://www.hangangholic.com/,2015 한강홀릭사무국 02-449-9060,"- 이색레저스포츠체험: 열기구체험, 요트체험, 행글라이딩 체험, 유로번지 체험, 익...",NaN,NaN,...,126.98847306834401,37.50544760752040,NaN,http://place.map.kakao.com/24580371,2015,1,954978.60173327091616,1944947.40767574543133,954789.22465931111947,1945253.56959683378227
3,그랜드케이팝페스티벌,2015-09-04,2015-09-04,잠실 올림픽 주 경기장,대한민국의 최정상급 케이팝 스타들이 대거 출연하는 케이팝 한류 페스티벌이 9월4일 ...,http://www.grandkpopfestival.com,"한국관광공사 관리자 033-738-3224, 3226",NaN,NaN,NaN,...,127.07302486580099,37.51459061509421,서울 송파구 올림픽로 25,http://place.map.kakao.com/8530688,2015,1,962456.31154805736151,1945924.62128509767354,962266.88981846836396,1946230.66387243219651
4,서리풀페스티벌,2015-09-15,2015-09-20,예술의 전당,서리풀페스티벌은 서초구의 고품격 예술자원과 글로벌 문화콘텐츠를 결집한 축제로 K-컬...,http://blog.naver.com/sccf150508,"서리풀페스티벌 조직위원회 관리자 02-2155-8212, 3477-2856",[서초 명사와 함께 하는 토요벼룩시장]\r - 일시 및 장소 : 9.19(토) 1...,NaN,NaN,...,127.01362582484900,37.47943350201761,서울 서초구 남부순환로 2406,http://place.map.kakao.com/25466462,2015,6,957186.86478320753668,1942049.53332294733264,956997.42696985078510,1942355.68534320360050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,세종탄신기념문화행사,2016-05-15,2016-05-15,국립한글박물관,제 619돌을 맞은 세종대왕 탄신 기념 문화행사가 5월 15일 국립한글박물관 잔디마...,http://www.hangeul.go.kr/event/userEventApplyD...,02-2124-6200,"- 공연 \r • 음악극 ‘위대한 성군, 세종’\r• 음악을 사랑한 세종(궁중음...",NaN,NaN,...,126.98097747372800,37.52118537749170,서울 용산구 서빙고로 139,http://place.map.kakao.com/25612651,2016,1,954325.70704925048631,1946697.05184654076584,954136.36129998799879,1947003.20955536048859
72,희망의운동화나눔축제,2016-05-20,2016-05-21,서울 광화문 광장,<희망의 운동화 나눔 축제>는 운동화 위에 세계 평화와 희망을 상징하는 이미지를 그...,http://www.mizy.net,서울시립청소년문화교류센터(Mizy) 관리자 미지(Mizy)센터 02-755-1024,NaN,기간내자유,NaN,...,126.97691639601514,37.57260286081019,NaN,http://place.map.kakao.com/8193961,2016,2,953998.49637373327278,1952403.57904530502856,953809.23853273945861,1952709.69590329681523
73,빗물축제,2016-07-28,2016-07-31,서울시청 광장,물순환 시민 문화제에서 서울시와 시민들의 실천의지가 함께 할 수 있는 축제의 장으로...,https://www.facebook.com/events/1126526240721644,관리자 물순환정책과 02-2133-3762,"빗물 퍼포먼스(플레쉬몹), , 물순환 박람회, 레인마켓, 빗물놀이터, 물순환 시민참...",12시간,NaN,...,126.97796892695699,37.56567260428050,NaN,http://place.map.kakao.com/7940285,2016,4,954087.18129648745526,1951634.17902306932956,953897.91124661837239,1951940.30073793302290
74,남산골한옥마을세시맞이오(五)대감설날잔치,2016-02-08,2016-02-09,서울특별시 중구 퇴계로34길 28,서울 중구에 위치한 남산골한옥마을에서는 설 명절을 맞아 다양한 전통문화 프로그램을 ...,http://hanokmaeul.or.kr,02-2264-4412,NaN,기간 내 자유,NaN,...,126.99475754194501,37.55927386383180,서울 중구 퇴계로34길 28,http://place.map.kakao.com/8246127,2016,2,955566.04916115337983,1950916.18382008699700,955376.75659718934912,1951222.28919856110588


In [12]:
tmp.to_excel("festival_info_1516_transform_coord.xlsx", encoding='cp949', index=False, float_format="%.14f")